# Processing of Sentinel-1 SAR images 
# Application to flood monitoring

# 1. Single image processing

## 1.1 Settings

In [ ]:
in_dir='~/SAR_DATA/TP_S1/InputData'
!ls $in_dir

The Sentinel-1 images are stored in the folders:
TP_S1/InputData/S1B_IW_GRDH_1SDV_20181004T060051_20181004T060116_023982_029EA5_0956.S
AFE/measurement/
TP_S1/InputData/S1A_IW_GRDH_1SDV_20181016T060051_20181016T060116_024157_02A459_BFCA.
SAFE/measurement/

In [ ]:
im1_name="S1A_IW_GRDH_1SDV_20181004T060051_20181004T060116_023982_029EA5_0956.SAFE"
tiff_vv1="s1a-iw-grd-vv-20181004t060051-20181004t060116-023982-029ea5-001.tiff"

In [ ]:
proc_dir='~/SAR_DATA/TP_S1/OutputData'
!ls $proc_dir

##   1.2 Radiometric Calibration: from DN to $\sigma_0$

In [ ]:
!otbcli_SARCalibration -in $in_dir/$im1_name/measurement/$tiff_vv1 -out $proc_dir/s1a-iw-grd-vv-20181004-sigma0.tiff -lut sigma

## 1.3 Clipping: selection of a sub-image

In [ ]:
!pwd
#!gdalwarp -s_srs EPSG:4326 -t_srs EPSG:4326 -te 2.11 42.89 3.04 43.54 -of GTiff s1a_iw_grd_vv_20181004_sigma0.tiff s1a-iw-grd-vh-20181004-sigma0clipped.tiff
#France: 46°00'N, 2°00'E 

!otbcli_ExtractROI -in $proc_dir/s1a-iw-grd-vv-20181004-sigma0.tiff  -mode extent -mode.extent.unit lonlat -mode.extent.ulx 2.11 -mode.extent.uly 42.89 -mode.extent.lrx 3.04 -mode.extent.lry 43.54 -out $proc_dir/s1a-iw-grd-vv-20181004-sigma0Clip.tiff

## 1.4 Speckle filtering

In [ ]:
!otbcli_Despeckle -in $proc_dir/s1a-iw-grd-vv-20181004-sigma0Clip.tiff -out $proc_dir/s1a-iw-grd-vv-20181004-sigma0ClipFilt.tiff -filter frost -filter.frost.rad 3 -filter.frost.deramp 0.2

# 1.5 Filtered image orthorectification

In [ ]:
!otbcli_OrthoRectification -io.in $proc_dir/s1a-iw-grd-vv-20181004-sigma0ClipFilt.tiff -io.out $proc_dir/s1a-iw-grd-vv-20181004-sigma0ClipFiltOrtho.tiff -map wgs -opt.gridspacing 4 -elev.dem $in_dir/SRTM/ -elev.geoid $in_dir/Geoid/egm96.grd

# 1.6 Superimpose a Sentinel-2 image

In [ ]:
overlay_im=in_dir+"/SENTINEL2A_20181025-104115-456_L2A_T31TDH_D_V1-9/SENTINEL2A_20181025-104115-456_L2A_T31TDH_D_V1-9_FRE_B3.tif"
!otbcli_Superimpose -inr $proc_dir/s1a-iw-grd-vv-20181004-sigma0ClipFiltOrtho.tiff -inm $overlay_im  -out $proc_dir/S2superonS1.tiff

# 2 Batch processing: VV and HV images at both dates

In [ ]:
import os

in_dir='~/SAR_DATA/TP_S1/InputData'
proc_dir='~/SAR_DATA/TP_S1/OutputData'
im1_name="S1A_IW_GRDH_1SDV_20181004T060051_20181004T060116_023982_029EA5_0956.SAFE"
im2_name="S1A_IW_GRDH_1SDV_20181016T060051_20181016T060116_024157_02A459_BFCA.SAFE"
tiff_vv1="s1a-iw-grd-vv-20181004t060051-20181004t060116-023982-029ea5-001.tiff"
tiff_vh1="s1a-iw-grd-vh-20181004t060051-20181004t060116-023982-029ea5-002.tiff"
tiff_vv2="s1a-iw-grd-vv-20181016t060051-20181016t060116-024157-02a459-001.tiff"
tiff_vh2="s1a-iw-grd-vh-20181016t060051-20181016t060116-024157-02a459-002.tiff"
#---------------------------------------#
#remove old results before processing
#---------------------------------------#
files=os.listdir(os.path.expanduser(proc_dir))

for ff in files:
    os.remove(os.path.join(os.path.expanduser(proc_dir),ff))
 
imlist=['s1a-iw-grd-vv-20181004', 's1a-iw-grd-vh-20181016', 's1a-iw-grd-vh-20181004', 's1a-iw-grd-vv-20181016']
print(imlist)

In [ ]:
imlist=['s1a-iw-grd-vv-20181004', 's1a-iw-grd-vh-20181016', 's1a-iw-grd-vh-20181004', 's1a-iw-grd-vv-20181016']

In [ ]:
!otbcli_SARCalibration -in $in_dir/$im1_name/measurement/$tiff_vv1 -out $proc_dir/s1a-iw-grd-vv-20181004-sigma0.tiff -lut sigma
!otbcli_SARCalibration -in $in_dir/$im1_name/measurement/$tiff_vh1 -out $proc_dir/s1a-iw-grd-vh-20181004-sigma0.tiff -lut sigma
!otbcli_SARCalibration -in $in_dir/$im2_name/measurement/$tiff_vv2 -out $proc_dir/s1a-iw-grd-vv-20181016-sigma0.tiff -lut sigma
!otbcli_SARCalibration -in $in_dir/$im2_name/measurement/$tiff_vh2 -out $proc_dir/s1a-iw-grd-vh-20181016-sigma0.tiff -lut sigma

In [ ]:
for im in imlist:
    #print(len(file))
    im_name=im+'-sigma0.tiff'
    clip_name=im+'-sigma0Clip.tiff'
    filt_name=im+'-sigma0ClipFilt.tiff'
    ortho_name=im+'-sigma0ClipFiltOrth.tiff'
    !otbcli_ExtractROI -in $proc_dir/$im_name -mode extent -mode.extent.unit lonlat -mode.extent.ulx 2.11 -mode.extent.uly 42.89 -mode.extent.lrx 3.04 -mode.extent.lry 43.54 -out $proc_dir/$clip_name
    !otbcli_Despeckle -in $proc_dir/$clip_name -out $proc_dir/$filt_name -filter frost -filter.frost.rad 3 -filter.frost.deramp 0.2
    !otbcli_OrthoRectification -io.in $proc_dir/$filt_name -io.out $proc_dir/$ortho_name -map wgs -opt.gridspacing 4 -elev.dem $in_dir/SRTM/ -elev.geoid $in_dir/Geoid/egm96.grd


In [ ]:
overlay_im=in_dir+"/SENTINEL2A_20181025-104115-456_L2A_T31TDH_D_V1-9/SENTINEL2A_20181025-104115-456_L2A_T31TDH_D_V1-9_FRE_B3.tif"
!otbcli_Superimpose -inr $proc_dir/$ortho_name -inm $overlay_im  -out $proc_dir/S2superonS1.tiff